In [1]:
# import web driver
from selenium import webdriver

# create ChromeOptions object
chrome_options = webdriver.ChromeOptions()

# specifies the path to the chromedriver.exe
driver = webdriver.Chrome(options=chrome_options)

# driver.get method() will navigate to a page given by the URL address
driver.get('http://www.linkedin.com')


In [2]:
from selenium.webdriver.common.by import By
from dotenv import load_dotenv
import os

load_dotenv()

linkedin_password = os.getenv('linkedin_pw')
linkedin_username = os.getenv('linkedin_uid')

# locate email input field by id
email_input = driver.find_element(By.ID, 'session_key')

# send_keys() to simulate key strokes
email_input.send_keys(linkedin_username)

# locate password input field by id
password_input = driver.find_element(By.ID, 'session_password')

# send_keys() to simulate key strokes
password_input.send_keys(linkedin_password)

In [3]:
from selenium.webdriver.common.by import By

# Locate submit button by class name
log_in_button = driver.find_element(By.CLASS_NAME, 'sign-in-form__submit-btn--full-width')

# Click the button
log_in_button.click()


In [4]:
from selenium.webdriver.common.by import By

# Locate the search button by class name
search_button = driver.find_element(By.CLASS_NAME, 'search-global-typeahead__collapsed-search-button')

# Click the button
search_button.click()


In [5]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# Locate the search input field by class name
search_input = driver.find_element(By.CLASS_NAME, 'search-global-typeahead__input')

# Clear any existing text in the search input
search_input.clear()

# Send the search query "Software Developer"
search_input.send_keys('Software Developer')

# Submit the search form by pressing Enter
search_input.send_keys(Keys.RETURN)


In [6]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Wait for the button with the text "People" to be present
button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//button[text()="People"]'))
)

# Click the button
button.click()


In [7]:
# Assuming you have already navigated to LinkedIn, logged in, and performed the search
import csv
from selenium.common.exceptions import StaleElementReferenceException

# Wait for the search results to be present
search_results = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, '//li[@class="reusable-search__result-container"]'))
)

# List to store extracted data
data_list = []

# Iterate through the search results and extract data
for result in search_results:
    try:
        # Wait for name element to be present
        name_element = WebDriverWait(result, 5).until(
            EC.presence_of_element_located((By.XPATH, './/span[@class="text-heading-xlarge inline t-24 v-align-middle break-words"]'))
        )
        name = name_element.text.strip()

        # Extract current position
        current_position = result.find_element(By.XPATH, './/div[@class="entity-result__primary-subtitle"]').text.strip()

        # Extract LinkedIn URL
        linkedin_url = result.find_element(By.XPATH, './/a[@class="app-aware-link"]').get_attribute('href')

        # Append data to the list
        data_list.append([name, current_position, linkedin_url])
    except StaleElementReferenceException:
        # Retry the operation if the element becomes stale
        continue
    except Exception as e:
        print(f"Error extracting data: {str(e)}")

# Save data to CSV file
csv_filename = 'linkedin_data.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    # Write header
    csv_writer.writerow(['Name', 'Current Position', 'LinkedIn URL'])
    # Write data
    csv_writer.writerows(data_list)

print(f"Data saved to {csv_filename}")


Error extracting data: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF64B8B7012+3522402]
	(No symbol) [0x00007FF64B4D8352]
	(No symbol) [0x00007FF64B385ABB]
	(No symbol) [0x00007FF64B3CBF0E]
	(No symbol) [0x00007FF64B3CC08C]
	(No symbol) [0x00007FF64B3C12AC]
	(No symbol) [0x00007FF64B3EF09F]
	(No symbol) [0x00007FF64B3C120A]
	(No symbol) [0x00007FF64B3EF270]
	(No symbol) [0x00007FF64B40BDA3]
	(No symbol) [0x00007FF64B3EEE03]
	(No symbol) [0x00007FF64B3BF4D4]
	(No symbol) [0x00007FF64B3C05F1]
	GetHandleVerifier [0x00007FF64B8E9B9D+3730157]
	GetHandleVerifier [0x00007FF64B93F02D+4079485]
	GetHandleVerifier [0x00007FF64B9375D3+4048163]
	GetHandleVerifier [0x00007FF64B60A649+718233]
	(No symbol) [0x00007FF64B4E4A3F]
	(No symbol) [0x00007FF64B4DFA94]
	(No symbol) [0x00007FF64B4DFBC2]
	(No symbol) [0x00007FF64B4CF2E4]
	BaseThreadInitThunk [0x00007FFAD825257D+29]
	RtlUserThreadStart [0x00007FFAD9A6AA58+40]

Error extracting data: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF64B8B701

In [8]:
import psycopg2
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
from dotenv import load_dotenv
import time

# Connect to your PostgreSQL database using the configuration
load_dotenv()

conn = psycopg2.connect(
    dbname=os.getenv('db_name'),
    user=os.getenv('db_user'),
    password=os.getenv('db_password'),
    host=os.getenv('db_host'),
    port=os.getenv('db_port')
)

# Create a cursor
cursor = conn.cursor()

# Assume you have navigated to the page with the search results
# Introduce a brief delay to allow dynamic content to load
time.sleep(2)

# Wait for the search results to be present
search_results = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, '//li[@class="reusable-search__result-container"]'))
)

# Iterate through the search results and extract data
for result in search_results:
    try:
        # Wait for name element to be present
        name_element = WebDriverWait(result, 5).until(
            EC.presence_of_element_located((By.XPATH, './/span[@class="text-heading-xlarge inline t-24 v-align-middle break-words"]'))
        )

        # Extract data from each search result
        name = name_element.text.strip()

        # Extract current position
        current_position = result.find_element(By.XPATH, './/div[@class="entity-result__primary-subtitle"]').text.strip()

        # Extract skills
        skills = result.find_element(By.XPATH, './/div[@class="entity-result__insights"]//a').text.strip()

        # Extract LinkedIn URL
        linkedin_url = result.find_element(By.XPATH, './/a[@class="app-aware-link"]').get_attribute('href')

        # Insert data into PostgreSQL
        cursor.execute("""
            INSERT INTO your_table_name (name, current_position, skills, linkedin_url)
            VALUES (%s, %s, %s, %s)
        """, (name, current_position, skills, linkedin_url))

        # Commit the transaction
        conn.commit()
    except Exception as e:
        print(f"Error extracting or saving data: {str(e)}")

# Close the cursor and connection
cursor.close()
conn.close()


Error extracting or saving data: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF64B8B7012+3522402]
	(No symbol) [0x00007FF64B4D8352]
	(No symbol) [0x00007FF64B385ABB]
	(No symbol) [0x00007FF64B3CBF0E]
	(No symbol) [0x00007FF64B3CC08C]
	(No symbol) [0x00007FF64B3C12AC]
	(No symbol) [0x00007FF64B3EF09F]
	(No symbol) [0x00007FF64B3C120A]
	(No symbol) [0x00007FF64B3EF270]
	(No symbol) [0x00007FF64B40BDA3]
	(No symbol) [0x00007FF64B3EEE03]
	(No symbol) [0x00007FF64B3BF4D4]
	(No symbol) [0x00007FF64B3C05F1]
	GetHandleVerifier [0x00007FF64B8E9B9D+3730157]
	GetHandleVerifier [0x00007FF64B93F02D+4079485]
	GetHandleVerifier [0x00007FF64B9375D3+4048163]
	GetHandleVerifier [0x00007FF64B60A649+718233]
	(No symbol) [0x00007FF64B4E4A3F]
	(No symbol) [0x00007FF64B4DFA94]
	(No symbol) [0x00007FF64B4DFBC2]
	(No symbol) [0x00007FF64B4CF2E4]
	BaseThreadInitThunk [0x00007FFAD825257D+29]
	RtlUserThreadStart [0x00007FFAD9A6AA58+40]

Error extracting or saving data: Message: 
Stacktrace:
	GetHandleVerifie